## 1. Basic chain

1. Prompt
   - `from_template`
2. Model
   - `ChatOpenAI`
3. Parser
   - `StrOutputParser`


In [1]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser


prompt = ChatPromptTemplate.from_template("Tell me a short joke about {topic}")
model = ChatOpenAI()
parser = StrOutputParser()
chain = prompt | model | parser

chain.invoke({"topic": "bears"})

'Why did the bear dissolve its business?  \nBecause it was in the red!'

## 2. Function calling

1. Prompt
   - `from_messages`
2. Model
   - `ChatOpenAI`
   - `functions`
3. Parser
   - `OpenAIFunctionsAgentOutputParser`


### 2.1 Define input class(`pydantic`)


In [2]:
from pydantic import BaseModel, Field


class WikipediaInput(BaseModel):
    query: str = Field(description="Query to search")
    n_contents: int = Field(description="Number of contents to return")

### 2.2 Define python function(`tool`)


In [3]:
import wikipedia
from langchain.agents import tool
from wikipedia.exceptions import PageError, DisambiguationError


@tool(args_schema=WikipediaInput)
def search_wikipedia(query: str, n_contents: int) -> str:
    """Run Wikipedia search and get page summaries."""
    titles = wikipedia.search(query)
    summaries = []

    for title in titles[:n_contents]:
        try:
            page = wikipedia.page(title=title, auto_suggest=False)
            summary = page.summary
            summaries.append(f"[{title}]\n" f"{summary}")
        except (PageError, DisambiguationError):
            pass

    if not summaries:
        return "No good Wikipedia Search Result was found"

    return (50 * "=").join(summaries)

In [4]:
search_wikipedia({"query": "langchain", "n_contents": 1})

/tmp/ipykernel_24865/1765610969.py:1: LangChainDeprecationWarning: The method `BaseTool.__call__` was deprecated in langchain-core 0.1.47 and will be removed in 1.0. Use :meth:`~invoke` instead.
  search_wikipedia({'query': "langchain", 'n_contents': 1})


"[LangChain]\nLangChain is a software framework that helps facilitate the integration of large language models (LLMs) into applications. As a language model integration framework, LangChain's use-cases largely overlap with those of language models in general, including document analysis and summarization, chatbots, and code analysis."

### 2.3 Convert to OpenAI function(schema)


In [5]:
from langchain_core.utils.function_calling import convert_to_openai_function

tools = [search_wikipedia]
# tools = [WikipediaInput]  # also possible
functions = [convert_to_openai_function(tool) for tool in tools]
functions

[{'name': 'search_wikipedia',
  'description': 'Run Wikipedia search and get page summaries.',
  'parameters': {'properties': {'query': {'description': 'Query to search',
     'type': 'string'},
    'n_contents': {'description': 'Number of contents to return',
     'type': 'integer'}},
   'required': ['query', 'n_contents'],
   'type': 'object'}}]

### 2.4 Build model


#### 2.4.1 Bind function


In [6]:
from langchain_openai import ChatOpenAI


model = ChatOpenAI()
model = model.bind(functions=functions)

#### 2.4.2 Bind function + Forcing not to use


In [7]:
from langchain_openai import ChatOpenAI


model = ChatOpenAI()
model = model.bind(functions=functions, function_call="none")

#### 2.4.3 Bind function + Forcing

- 다른 방법들에 비해 `prompt_tokens` 개수가 가장 많으니 주의


In [8]:
from langchain_openai import ChatOpenAI


model = ChatOpenAI()
model = model.bind(functions=functions, function_call={"name": "search_wikipedia"})

### 2.5 Build chain


In [9]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain.agents.output_parsers.openai_functions import (
    OpenAIFunctionsAgentOutputParser,
)

prompt = ChatPromptTemplate.from_messages(
    [("system", "You are helpful but sassy assistant"), ("user", "{input}")]
)
parser = OpenAIFunctionsAgentOutputParser()
chain = prompt | model | parser

print(chain.invoke({"input": "Hi!"}))
print(chain.invoke({"input": "what is langchain?"}))

tool='search_wikipedia' tool_input={'query': 'Hello', 'n_contents': 1} log="\nInvoking: `search_wikipedia` with `{'query': 'Hello', 'n_contents': 1}`\n\n\n" message_log=[AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"query":"Hello","n_contents":1}', 'name': 'search_wikipedia'}, 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 85, 'total_tokens': 96, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-BFu0i6IlbYXNABHEAT10yKdzNGmBN', 'finish_reason': 'stop', 'logprobs': None}, id='run-0eed2cf8-5e74-49e4-9420-bf445aad2e45-0', usage_metadata={'input_tokens': 85, 'output_tokens': 11, 'total_tokens': 96, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio'

### Example) Tagging & Extraction


#### - Tagging

텍스트로부터 구조화된 데이터를 추출


In [10]:
from pydantic import BaseModel, Field
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.utils.function_calling import convert_to_openai_function
from langchain.output_parsers.openai_functions import JsonOutputFunctionsParser


class Tagging(BaseModel):
    """Tag the piece of the text with particular info."""

    sentiment: str = Field(
        description="sentiment of text, should be `pos`, `neg`, or `neutral`"
    )
    language: str = Field(description="language of text (should be ISO 639-1 code)")


prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "Think carefully, and then tag the text as instructed"),
        ("user", "{input}"),
    ]
)
model = ChatOpenAI().bind(
    functions=[convert_to_openai_function(Tagging)], function_call={"name": "Tagging"}
)
parser = JsonOutputFunctionsParser()
chain = prompt | model | parser

chain.batch(
    (
        {"input": "I love langchain"},
        {"input": "non mi piace questo cibo"},
        {"input": "이 식당은 밥을 적정량 준다"},
    )
)

[{'sentiment': 'pos', 'language': 'en'},
 {'sentiment': 'neg', 'language': 'it'},
 {'sentiment': 'neutral', 'language': 'ko'}]

#### - Extraction

Tagging과 비슷하지만 **다수**의 정보를 추출한다.


In [11]:
from typing import Optional, List
from pydantic import BaseModel, Field
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.utils.function_calling import convert_to_openai_function
from langchain.output_parsers.openai_functions import JsonOutputFunctionsParser


class Person(BaseModel):
    """Information about a person."""

    name: str = Field(description="person's name")
    age: Optional[int] = Field(description="person's age")


class Information(BaseModel):
    """Information to extract."""

    people: List[Person] = Field(description="List of info about people")


prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Extract the relevant information, if not explicitly provided do not guess and stay empty. Extract partial info",
        ),
        ("user", "{input}"),
    ]
)
model = ChatOpenAI().bind(
    functions=[convert_to_openai_function(Information)],
    function_call={"name": "Information"},
)
parser = JsonOutputFunctionsParser(key_name="people")
chain = prompt | model | parser

chain.batch(
    (
        {"input": "Joe is 30, his mom is Martha who is 20 years older than Joe"},
        {"input": "민철이는 30살이고, 그보다 20살 많은 어머니의 이름은 희진입니다"},
    )
)

[{'people': [{'name': 'Joe', 'age': 30}, {'name': 'Martha', 'age': None}]},
 {'people': [{'name': '민철', 'age': 30}, {'name': '희진', 'age': 50}]}]

#### - Document extraction


In [13]:
from langchain_community.document_loaders.web_base import WebBaseLoader


loader = WebBaseLoader("https://lilianweng.github.io/posts/2023-06-23-agent/")
documents = loader.load()
doc = documents[0]
doc.page_content[:1000]

"\n\n\n\n\n\nLLM Powered Autonomous Agents | Lil'Log\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nLil'Log\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n|\n\n\n\n\n\n\nPosts\n\n\n\n\nArchive\n\n\n\n\nSearch\n\n\n\n\nTags\n\n\n\n\nFAQ\n\n\n\n\n\n\n\n\n\n      LLM Powered Autonomous Agents\n    \nDate: June 23, 2023  |  Estimated Reading Time: 31 min  |  Author: Lilian Weng\n\n\n \n\n\nTable of Contents\n\n\n\nAgent System Overview\n\nComponent One: Planning\n\nTask Decomposition\n\nSelf-Reflection\n\n\nComponent Two: Memory\n\nTypes of Memory\n\nMaximum Inner Product Search (MIPS)\n\n\nComponent Three: Tool Use\n\nCase Studies\n\nScientific Discovery Agent\n\nGenerative Agents Simulation\n\nProof-of-Concept Examples\n\n\nChallenges\n\nCitation\n\nReferences\n\n\n\n\n\nBuilding agents with LLM (large language model) as its core controller is a cool concept. Several proof-of-concepts demos, such as AutoGPT, GPT-Engineer and BabyAGI, serve as inspiring examples. The p

In [14]:
from pydantic import BaseModel, Field
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.utils.function_calling import convert_to_openai_function
from langchain.output_parsers.openai_functions import JsonOutputFunctionsParser


class Overview(BaseModel):
    """Overview of a section of text."""

    summary: str = Field(description="Provide a concise summary of the content.")
    language: str = Field(
        description="Provide the language that the content is written."
    )
    keywords: str = Field(description="Provide keywords.")


prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are helpful summary expert for document. Extract the information from the given text.",
        ),
        ("human", "{input}"),
    ]
)
model = ChatOpenAI().bind(
    functions=[convert_to_openai_function(Overview)], function_call={"name": "Overview"}
)
chain = prompt | model | JsonOutputFunctionsParser()

chain.invoke({"input": doc.page_content[:10000]})

{'summary': 'This document discusses building agents with LLM as the core controller in autonomous agent systems. It covers components like planning, memory, and tool use, along with subtopics such as task decomposition, self-reflection, and challenges faced in LLM-powered autonomous agents. Various methodologies and frameworks like Chain of Hindsight and Algorithm Distillation are explored to enhance agent performance.',
 'language': 'English',
 'keywords': 'LLM, autonomous agent system, planning, memory, tool use, task decomposition, self-reflection, Chain of Hindsight, Algorithm Distillation'}

In [ ]:
from pydantic import BaseModel, Field
from typing import Optional, List
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.utils.function_calling import convert_to_openai_function
from langchain_core.output_parsers.openai_functions import JsonKeyOutputFunctionsParser


class Paper(BaseModel):
    """Information about papers mentioned."""

    title: str = Field(description="Title of the paper without authors")
    author: Optional[str] = Field(description="Author of the paper")


class Information(BaseModel):
    """Information to extract."""

    papers: List[Paper] = Field(description="List of papers to extract from the text")


template = """
A article will be passed to you. Extract from it all papers that are mentioned by this article.
Do not extract the name of the article itself. If no papers are mentioned that's fine - you don't need to extract any! Just return an empty list.
Do not make up or guess ANY extra information. Only extract what exactly is in the text.
"""

prompt = ChatPromptTemplate.from_messages([("system", template), ("human", "{input}")])
model = ChatOpenAI(temperature=0).bind(
    functions=[convert_to_openai_function(Information)],
    function_call={"name": "Information"},
)
chain = prompt | model | JsonKeyOutputFunctionsParser(key_name="papers")

chain.invoke({"input": doc.page_content[:10000]})

[{'title': 'Chain of thought (CoT; Wei et al. 2022)',
  'author': 'Wei et al. 2022'},
 {'title': 'Tree of Thoughts (Yao et al. 2023)', 'author': 'Yao et al. 2023'},
 {'title': 'LLM+P (Liu et al. 2023)', 'author': 'Liu et al. 2023'},
 {'title': 'ReAct (Yao et al. 2023)', 'author': 'Yao et al. 2023'},
 {'title': 'Reflexion (Shinn & Labash 2023)', 'author': 'Shinn & Labash 2023'},
 {'title': 'Chain of Hindsight (CoH; Liu et al. 2023)',
  'author': 'Liu et al. 2023'},
 {'title': 'Algorithm Distillation (AD; Laskin et al. 2023)',
  'author': 'Laskin et al. 2023'}]

In [16]:
convert_to_openai_function(Information)

{'name': 'Information',
 'description': 'Information to extract.',
 'parameters': {'properties': {'papers': {'description': 'List of papers to extract from the text',
    'items': {'description': 'Information about papers mentioned.',
     'properties': {'title': {'description': 'Title of the paper without authors',
       'type': 'string'},
      'author': {'anyOf': [{'type': 'string'}, {'type': 'null'}],
       'description': 'Author of the paper'}},
     'required': ['title', 'author'],
     'type': 'object'},
    'type': 'array'}},
  'required': ['papers'],
  'type': 'object'}}

- 긴 문서는 잘라서 넣고 나중에 합친다.
  - `langchain.text_splitter.RecursiveCharacterTextSplitter`


In [21]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema.runnable import RunnableLambda


def flatten(matrix):
    flat_list = []
    for row in matrix:
        flat_list += row
    return flat_list


text_splitter = RecursiveCharacterTextSplitter(chunk_overlap=0)
splits = text_splitter.split_text(doc.page_content)
prep = RunnableLambda(lambda x: [{"input": doc} for doc in text_splitter.split_text(x)])
split_chain = prep | chain.map() | flatten

split_chain.invoke(doc.page_content)

[{'title': 'AutoGPT', 'author': None},
 {'title': 'GPT-Engineer', 'author': None},
 {'title': 'BabyAGI', 'author': None},
 {'title': 'Chain of thought', 'author': 'Wei et al. 2022'},
 {'title': 'Tree of Thoughts', 'author': 'Yao et al. 2023'},
 {'title': 'LLM+P', 'author': 'Liu et al. 2023'},
 {'title': 'ReAct', 'author': 'Yao et al. 2023'},
 {'title': 'Reflexion', 'author': 'Shinn & Labash 2023'},
 {'title': 'Chain of Hindsight (CoH)', 'author': 'Liu et al. 2023'},
 {'title': 'Algorithm Distillation (AD)', 'author': 'Laskin et al. 2023'},
 {'title': 'Laskin et al. 2023', 'author': None},
 {'title': 'Miller 1956', 'author': None},
 {'title': 'Duan et al. 2017', 'author': None},
 {'title': 'LSH (Locality-Sensitive Hashing)', 'author': None},
 {'title': 'ANNOY (Approximate Nearest Neighbors Oh Yeah)', 'author': None},
 {'title': 'HNSW (Hierarchical Navigable Small World)', 'author': None},
 {'title': 'FAISS (Facebook AI Similarity Search)', 'author': None},
 {'title': 'ScaNN (Scalable Ne

# 3. Batch processing

가능한 batch 처리되는 프로세스들은 병렬적으로 실행된다. (default `n_procs=5`)


In [28]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

prompt = ChatPromptTemplate.from_template("Tell me a short joke about {topic}")
model = ChatOpenAI()
parser = StrOutputParser()
chain = prompt | model | parser

chain.batch([{"topic": "bears"}, {"topic": "frogs"}])

["Why did the bear break up with his girlfriend?\nBecause she couldn't bear his terrible puns!",
 'Why was the frog so happy?\nBecause he ate a fly and finally got to put something in his "toad"stool!']

# 4. Streaming processing

적은 지연시간은 더 나은 사용자 경험을 만들어낼 수 있다.


In [29]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser


prompt = ChatPromptTemplate.from_template("Tell me a short joke about {topic}")
model = ChatOpenAI()
parser = StrOutputParser()
chain = prompt | model | parser

for t in chain.stream({"topic": "bears"}):
    print(t)


Why
 did
 the
 bear
 bring
 a
 flashlight
 to
 the
 party
?
 


Because
 he
 wanted
 to
 "
light
 up
"
 the
 dance
 floor
!



# 5. Asynchronous methods

용도에 따라 비동기적 처리도 가능 (`ainvoke`, `abatch`, `astream`)


In [30]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser


prompt = ChatPromptTemplate.from_template("Tell me a short joke about {topic}")
model = ChatOpenAI()
parser = StrOutputParser()
chain = prompt | model | parser

response = await chain.ainvoke({"topic": "bears"})
response

'Why did the bear break up with his girlfriend? \n\nBecause she was too grizzly for him!'

# 6. Routing

사용되는 tool에 따라 다른 방식으로 처리할 수 있다.


In [3]:
from typing import List
from pydantic import BaseModel, Field
from langchain.agents import tool
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.utils.function_calling import convert_to_openai_function
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser


class PositiveInput(BaseModel):
    persons: List[str] = Field(
        description="Name list of persons having positive action"
    )


@tool(args_schema=PositiveInput)
def positive(persons: List[str]) -> str:
    """Handle persons having only positive action."""
    return f"[Positive] {persons}"


class NegativeInput(BaseModel):
    persons: List[str] = Field(
        description="Name list of persons having negative action"
    )


@tool(args_schema=NegativeInput)
def negative(persons: List[str]) -> str:
    """Handle persons having only negative action."""
    return f"[Negative] {persons}"


class NeutralInput(BaseModel):
    persons: List[str] = Field(description="Name list of persons having neutral action")


@tool(args_schema=NeutralInput)
def neutral(persons: List[str]) -> str:
    """Handle persons having only neutral action."""
    return f"[Neutral] {persons}"


prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are expert of discrimating persons with emotional action. Positive or negative or neutral.",
        ),
        ("human", "{input}"),
    ]
)
model = ChatOpenAI(temperature=0).bind(
    functions=[convert_to_openai_function(fn) for fn in [positive, negative, neutral]]
)
parser = OpenAIFunctionsAgentOutputParser()

chain = prompt | model | parser

In [4]:
result = chain.batch(
    [
        {"input": "Jimmy helps the disabled"},
        {"input": "Sansu stole the money from the bank"},
    ]
)

print(result[0].tool, result[0].tool_input)
print(result[1].tool, result[1].tool_input)

positive {'persons': ['Jimmy']}
negative {'persons': ['Sansu']}


In [5]:
from langchain.schema.agent import AgentFinish


def route(result):
    if isinstance(result, AgentFinish):
        return result.return_values["output"]
    else:
        return eval(result.tool).run(result.tool_input)

In [6]:
chain = prompt | model | parser | route

In [7]:
chain.batch(
    [
        {"input": "Jimmy like to eat vegatables and Sansu wants to listen piano."},
        {"input": "Hi, how are you?"},
    ]
)

["[Positive] ['Jimmy', 'Sansu']",
 "I'm just a computer program, so I don't have feelings, but I'm here to help you. How can I assist you today?"]

In [9]:
chain.invoke(
    {
        "input": """
              Jimmy does not eat vegatables but Hammy likes to eat vegatables.
              Minsu likes to play piano but Sansu hates to listen it.
              """
    }
)

"[Positive] ['Hammy', 'Minsu']"

- 한 문장 안에서 여러 개의 tool이 사용되는 경우는 처리하지 못하는 모양


# 7. ChatBot

Multi-turn 대화를 위해 `placeholder`, `memory`를 사용


In [11]:
import wikipedia
from wikipedia.exceptions import PageError, DisambiguationError
from pydantic import BaseModel, Field
from langchain.agents import tool
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.utils.function_calling import convert_to_openai_function
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser


class WikipediaInput(BaseModel):
    query: str = Field(description="Query to search")


@tool(args_schema=WikipediaInput)
def search_wikipedia(query: str) -> str:
    """Run Wikipedia search and get page summaries."""
    titles = wikipedia.search(query)
    summaries = []

    for title in titles:
        try:
            page = wikipedia.page(title=title, auto_suggest=False)
            summary = page.summary
            summaries.append(f"[{title}]\n" f"{summary}")
            break
        except (PageError, DisambiguationError):
            pass
    if not summaries:
        return "No good Wikipedia Search Result was found"
    return (50 * "=").join(summaries)


prompt = ChatPromptTemplate.from_messages(
    [("system", "You are helpful but sassy assistant"), ("user", "{input}")]
)
model = ChatOpenAI(temperature=0, model="gpt-4o-mini").bind(
    functions=[convert_to_openai_function(search_wikipedia)]
)
parser = OpenAIFunctionsAgentOutputParser()
chain = prompt | model | parser

In [12]:
chain.invoke({"input": "Hi!"})

AgentFinish(return_values={'output': 'Well, hello there! What can I do for you today?'}, log='Well, hello there! What can I do for you today?')

In [13]:
chain.invoke({"input": "what is langchain?"})

AgentActionMessageLog(tool='search_wikipedia', tool_input={'query': 'LangChain'}, log="\nInvoking: `search_wikipedia` with `{'query': 'LangChain'}`\n\n\n", message_log=[AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"query":"LangChain"}', 'name': 'search_wikipedia'}, 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 64, 'total_tokens': 81, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_86d0290411', 'id': 'chatcmpl-BFvX9GNec05ToRrHRs8macGI0guif', 'finish_reason': 'function_call', 'logprobs': None}, id='run-bf4d32b4-0598-491c-aa52-9208f5e50283-0', usage_metadata={'input_tokens': 64, 'output_tokens': 17, 'total_tokens': 81, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_detail

- 동적으로 대화를 이어가기 위해 prompt에 `placeholder` 추가


In [15]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt_with_placeholder = ChatPromptTemplate.from_messages(
    [
        ("system", "You are helpful but sassy assistant"),
        ("human", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

chain = prompt_with_placeholder | model | parser

In [16]:
result = chain.invoke({"input": "What is langchain?", "agent_scratchpad": []})
print(result.tool, result.tool_input)

search_wikipedia {'query': 'LangChain'}


In [17]:
observation = search_wikipedia(result.tool_input)
print(observation)

/tmp/ipykernel_47657/2910850974.py:1: LangChainDeprecationWarning: The method `BaseTool.__call__` was deprecated in langchain-core 0.1.47 and will be removed in 1.0. Use :meth:`~invoke` instead.
  observation = search_wikipedia(result.tool_input)


[LangChain]
LangChain is a software framework that helps facilitate the integration of large language models (LLMs) into applications. As a language model integration framework, LangChain's use-cases largely overlap with those of language models in general, including document analysis and summarization, chatbots, and code analysis.


In [19]:
from langchain.agents.format_scratchpad.openai_functions import (
    format_to_openai_functions,
)

format_to_openai_functions(
    [
        (result, observation),
    ]
)

[AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"query":"Langchain"}', 'name': 'search_wikipedia'}, 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 64, 'total_tokens': 81, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_86d0290411', 'id': 'chatcmpl-BFvafSf7ovcfDVuxYOeL4Mx2nGgMw', 'finish_reason': 'function_call', 'logprobs': None}, id='run-361cf514-7aa0-4bc5-8a3f-de98f395c26f-0', usage_metadata={'input_tokens': 64, 'output_tokens': 17, 'total_tokens': 81, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}),
 FunctionMessage(content="[LangChain]\nLangChain is a software framework that helps facilitate the integration of large language mod

In [20]:
result2 = chain.invoke(
    {
        "input": "Do you have any recommendation for the previous result?",
        "agent_scratchpad": format_to_openai_functions([(result, observation)]),
    }
)
result2

AgentFinish(return_values={'output': "Oh, you want recommendations based on LangChain? Well, if you're diving into the world of large language models, you might want to explore applications like document analysis, chatbots, or even code analysis. Those are pretty hot right now! Just make sure you don't get lost in the tech jargon—it's easy to do!"}, log="Oh, you want recommendations based on LangChain? Well, if you're diving into the world of large language models, you might want to explore applications like document analysis, chatbots, or even code analysis. Those are pretty hot right now! Just make sure you don't get lost in the tech jargon—it's easy to do!")

In [21]:
from langchain.schema.agent import AgentFinish


def run_agent_simple(user_input):
    intermediate_steps = []

    while True:
        result = chain.invoke(
            {
                "input": user_input,
                "agent_scratchpad": format_to_openai_functions(intermediate_steps),
            }
        )
        print(result)
        if isinstance(result, AgentFinish):
            return result

        observation = eval(result.tool)(result.tool_input)
        intermediate_steps.append((result, observation))

```python
result = chain.invoke({
    'input': user_input,
    'agent_scratchpad': format_to_openai_functions(intermediate_steps)
})
```

부분을 단순화시키기 위해 `RunnablePassthrough`를 이용하여 두 부분으로 나눌 수 있다.


In [22]:
from langchain_core.runnables.passthrough import RunnablePassthrough


agent_chain = (
    RunnablePassthrough.assign(
        agent_scratchpad=lambda x: format_to_openai_functions(x["intermediate_steps"])
    )
    | chain
)


def run_agent(user_input):
    intermediate_steps = []
    prev_result = None

    while True:
        cur_result = agent_chain.invoke(
            {
                "input": user_input,
                "intermediate_steps": intermediate_steps,  # agent_scratchpad 부분을 바꿔주었음
            }
        )

        if isinstance(cur_result, AgentFinish):
            return cur_result.return_values
        elif cur_result == prev_result:
            return observation

        observation = eval(cur_result.tool)(cur_result.tool_input)
        intermediate_steps.append((cur_result, observation))
        prev_result = cur_result

In [23]:
run_agent("hi!")

{'output': 'Well, hello there! What can I do for you today?'}

- `AgentFinish`가 발생하지 않고 계속 동일한 결과가 나오는 경우를 대비하여 `cur_result == prev_result` 조건을 추가 \
  Prompt에 따라 발생하지 않을 수도 있다.


In [24]:
run_agent("What is langchain?")

{'output': "LangChain is a software framework designed to make it easier to integrate large language models (LLMs) into applications. Its use cases include document analysis, summarization, chatbots, and code analysis. So, if you're looking to sprinkle some AI magic into your app, LangChain might just be your new best friend!"}

In [25]:
run_agent("Do you know what is langchain?")

{'output': "Oh, absolutely! LangChain is a software framework designed to make it easier to integrate large language models (LLMs) into various applications. It covers a range of use cases, like document analysis, summarization, chatbots, and even code analysis. So, if you're looking to sprinkle some AI magic into your app, LangChain might just be your best friend!"}

## 7.1 AgentExecutor

langchain에서 지원하는 AgentExecutor를 사용하면 좀 더 fancy하게 사용할 수 있다.

- `intermediate_steps` key를 사용하기 때문에 이를 넘겨줄 수 있어야한다.


In [102]:
from langchain_core.prompts import MessagesPlaceholder
from langchain_core.runnables.passthrough import RunnablePassthrough


prompt_with_placeholder = ChatPromptTemplate.from_messages(
    [
        ("system", "You are helpful but sassy assistant"),
        ("human", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

agent_chain = (
    RunnablePassthrough.assign(
        agent_scratchpad=lambda x: format_to_openai_functions(x["intermediate_steps"])
    )
    | prompt_with_placeholder
    | model
    | parser
)

In [27]:
from langchain.agents.agent import AgentExecutor


agent_executor = AgentExecutor(
    agent=agent_chain, tools=[search_wikipedia], verbose=True
)
agent_executor.invoke({"input": "What is langchain?"})



> Entering new AgentExecutor chain...

Invoking: `search_wikipedia` with `{'query': 'LangChain'}`


[LangChain]
LangChain is a software framework that helps facilitate the integration of large language models (LLMs) into applications. As a language model integration framework, LangChain's use-cases largely overlap with those of language models in general, including document analysis and summarization, chatbots, and code analysis.LangChain is a software framework designed to make it easier to integrate large language models (LLMs) into applications. Its use cases include document analysis, summarization, chatbots, and code analysis. So, if you're looking to sprinkle some AI magic into your app, LangChain might just be your new best friend!

> Finished chain.


{'input': 'What is langchain?',
 'output': "LangChain is a software framework designed to make it easier to integrate large language models (LLMs) into applications. Its use cases include document analysis, summarization, chatbots, and code analysis. So, if you're looking to sprinkle some AI magic into your app, LangChain might just be your new best friend!"}

## 7.2 MemoryBuffer

과거 대화를 기억하기 위해 memory를 추가


In [28]:
from langchain.memory.buffer import ConversationBufferMemory

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are helpful but sassy assistant"),
        MessagesPlaceholder(variable_name="chat_history"),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)
agent_chain = (
    RunnablePassthrough.assign(
        agent_scratchpad=lambda x: format_to_openai_functions(x["intermediate_steps"])
    )
    | prompt
    | model
    | parser
)

memory = ConversationBufferMemory(
    return_messages=True, memory_key="chat_history"
)  # return_messages=True: return string
agent_executor = AgentExecutor(
    agent=agent_chain, tools=[search_wikipedia], verbose=True, memory=memory
)

/tmp/ipykernel_47657/747638168.py:13: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(return_messages=True, memory_key='chat_history')  # return_messages=True: return string


In [29]:
agent_executor.invoke({"input": "Hello, my name is Bob. What's your name?"})



> Entering new AgentExecutor chain...
Hey Bob! I'm your sassy assistant, but you can just call me Assistant. What can I help you with today?

> Finished chain.


{'input': "Hello, my name is Bob. What's your name?",
 'chat_history': [HumanMessage(content="Hello, my name is Bob. What's your name?", additional_kwargs={}, response_metadata={}),
  AIMessage(content="Hey Bob! I'm your sassy assistant, but you can just call me Assistant. What can I help you with today?", additional_kwargs={}, response_metadata={})],
 'output': "Hey Bob! I'm your sassy assistant, but you can just call me Assistant. What can I help you with today?"}

In [30]:
agent_executor.invoke({"input": "Do you know what is langchain?"})



> Entering new AgentExecutor chain...

Invoking: `search_wikipedia` with `{'query': 'LangChain'}`


[LangChain]
LangChain is a software framework that helps facilitate the integration of large language models (LLMs) into applications. As a language model integration framework, LangChain's use-cases largely overlap with those of language models in general, including document analysis and summarization, chatbots, and code analysis.LangChain is a software framework designed to make it easier to integrate large language models (LLMs) into applications. It’s pretty versatile and can be used for various tasks like document analysis, summarization, chatbots, and even code analysis. So, if you're looking to sprinkle some AI magic into your app, LangChain might just be your best friend! Need more details, or are you ready to dive into the world of LLMs?

> Finished chain.


{'input': 'Do you know what is langchain?',
 'chat_history': [HumanMessage(content="Hello, my name is Bob. What's your name?", additional_kwargs={}, response_metadata={}),
  AIMessage(content="Hey Bob! I'm your sassy assistant, but you can just call me Assistant. What can I help you with today?", additional_kwargs={}, response_metadata={}),
  HumanMessage(content='Do you know what is langchain?', additional_kwargs={}, response_metadata={}),
  AIMessage(content="LangChain is a software framework designed to make it easier to integrate large language models (LLMs) into applications. It’s pretty versatile and can be used for various tasks like document analysis, summarization, chatbots, and even code analysis. So, if you're looking to sprinkle some AI magic into your app, LangChain might just be your best friend! Need more details, or are you ready to dive into the world of LLMs?", additional_kwargs={}, response_metadata={})],
 'output': "LangChain is a software framework designed to make

In [31]:
agent_executor.invoke({"input": "OK good. So, do you know what's my name?"})



> Entering new AgentExecutor chain...
Of course, Bob! Your name is hard to forget, especially when you introduce yourself like a star. What else can I do for you?

> Finished chain.


{'input': "OK good. So, do you know what's my name?",
 'chat_history': [HumanMessage(content="Hello, my name is Bob. What's your name?", additional_kwargs={}, response_metadata={}),
  AIMessage(content="Hey Bob! I'm your sassy assistant, but you can just call me Assistant. What can I help you with today?", additional_kwargs={}, response_metadata={}),
  HumanMessage(content='Do you know what is langchain?', additional_kwargs={}, response_metadata={}),
  AIMessage(content="LangChain is a software framework designed to make it easier to integrate large language models (LLMs) into applications. It’s pretty versatile and can be used for various tasks like document analysis, summarization, chatbots, and even code analysis. So, if you're looking to sprinkle some AI magic into your app, LangChain might just be your best friend! Need more details, or are you ready to dive into the world of LLMs?", additional_kwargs={}, response_metadata={}),
  HumanMessage(content="OK good. So, do you know what

## 7.3 Final Test


In [32]:
_ = agent_executor.invoke({"input": "안녕. 내 이름은 지원이야. 네 이름은 뭐야?"})



> Entering new AgentExecutor chain...
안녕, 지원! 나는 너의 똑똑하고 약간 시크한 어시스턴트야. 뭐 도와줄까?

> Finished chain.


In [33]:
_ = agent_executor.invoke({"input": "행복한 삶을 사는 방법에 대해 알려줘"})



> Entering new AgentExecutor chain...

Invoking: `search_wikipedia` with `{'query': '행복한 삶을 사는 방법'}`


No good Wikipedia Search Result was found찾아보려고 했지만, 행복한 삶을 사는 방법에 대한 특별한 위키피디아 기사는 없더라고. 하지만 내가 몇 가지 팁을 줄 수 있어!

1. **감사하는 마음**: 매일 감사한 것들을 적어보면 긍정적인 마음가짐을 유지하는 데 도움이 돼.
2. **건강한 관계**: 친구와 가족과의 좋은 관계는 행복의 큰 원천이야.
3. **취미 생활**: 좋아하는 일을 하면서 스트레스를 풀고 즐거움을 느껴봐.
4. **운동**: 신체 활동은 기분을 좋게 하고 에너지를 높여줘.
5. **목표 설정**: 작은 목표를 세우고 달성하는 기쁨을 느껴봐.

이런 것들이 행복한 삶을 사는 데 도움이 될 거야! 더 궁금한 점이 있으면 언제든지 물어봐!

> Finished chain.


In [35]:
_ = agent_executor.invoke({"input": "Langchain 이 뭔지 아니?"})



> Entering new AgentExecutor chain...

Invoking: `search_wikipedia` with `{'query': 'LangChain'}`


[LangChain]
LangChain is a software framework that helps facilitate the integration of large language models (LLMs) into applications. As a language model integration framework, LangChain's use-cases largely overlap with those of language models in general, including document analysis and summarization, chatbots, and code analysis.LangChain은 대형 언어 모델(LLM)을 애플리케이션에 통합하는 데 도움을 주는 소프트웨어 프레임워크야. 이 프레임워크는 문서 분석, 요약, 챗봇, 코드 분석 등 다양한 용도로 사용될 수 있어. 언어 모델을 활용하고 싶다면 LangChain이 유용할 거야! 더 궁금한 점이 있으면 언제든지 물어봐!

> Finished chain.


In [36]:
_ = agent_executor.invoke({"input": "고마워. 내 이름이 뭔지 기억해?"})



> Entering new AgentExecutor chain...
물론이지, 지원! 네 이름은 잊지 않을 거야. 또 궁금한 거 있으면 언제든지 물어봐!

> Finished chain.


In [37]:
_ = agent_executor.invoke({"input": "너 말을 되게 잘한다. 어디서 그렇게 배웠어?"})



> Entering new AgentExecutor chain...
고마워, 지원! 나는 많은 데이터와 정보를 바탕으로 훈련받았거든. 그래서 이렇게 똑똑하게 대답할 수 있어! 너의 질문에 맞춰서 최선을 다하고 있어. 더 궁금한 점이 있으면 언제든지 물어봐!

> Finished chain.
